In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd


In [2]:
model = load_model('./model/model-cnn-word2vec-3-classes-no-stemming.h5')

In [3]:
df = pd.read_csv('./dataset/dataset-testing/timnas_no_stemming.csv')

In [4]:
df

,full_text,processed
0,@safergieway @PSSI True bang. bener² ga ngerti...,"['true', 'bang', 'mengerti', 'mainnya', 'pssi'..."
1,Padahal buat jam terbang dan latihan kan kalo ...,"['jam', 'terbang', 'latihan', 'liga', 'klub', ..."
2,@garagarabola_ Lebih penting timnas bagus main...,"['timnas', 'bagus', 'mainnya', 'jersey', 'dike..."
3,@Tyows pdhal Pemain Timnas inggris nih gw rasa...,"['pemain', 'timnas', 'inggris', 'gue', 'enggak..."
4,@utdfocusid @bliblidotcom Semuanya out kecuali...,"['out', 'kecuali', 'hojlund', 'diganti', 'pema..."
...,...,...
1498,timnas malam ini mainnya gokil abizzzz 🇲🇨👊 #Ti...,"['timnas', 'malam', 'mainnya', 'gila', 'habis']"
1499,@Indostransfer Apresiasi utk ketahanan fisik t...,"['apresiasi', 'ketahanan', 'fisik', 'timnas', ..."
1500,90 menit full deg-degan... Babak pertama mainn...,"['menit', 'full', 'degdegan', 'babak', 'mainny..."
1501,"Sandy Walsh mainnya apik, solid. Beli jersey ...","['sandy', 'walsh', 'mainnya', 'apik', 'solid',..."


In [5]:
# join list of token as single document string
import ast

def join_text_list(texts):
    texts = ast.literal_eval(texts)
    return ' '.join([text for text in texts])
df["processed"] = df["processed"].apply(join_text_list)

df["processed"]

0       true bang mengerti mainnya pssi pelatih kelas ...
1       jam terbang latihan liga klub timnas mainnya t...
2                   timnas bagus mainnya jersey dikerenin
3       pemain timnas inggris gue enggak ada midfielde...
4       out kecuali hojlund diganti pemain timnas kcua...
                              ...                        
1498                      timnas malam mainnya gila habis
1499    apresiasi ketahanan fisik timnas sanggup berma...
1500    menit full degdegan babak mainnya bagus babak ...
1501    sandy walsh mainnya apik solid beli jersey tim...
1502    ruar timnas babak mainnya menurun meredam sera...
Name: processed, Length: 1503, dtype: object

In [6]:
# get tokenizer pickle
import pickle

with open('./tokenizer-3classes-no-stemming.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)


In [7]:
# # Parameters
# 
vocab_size = 20000
max_length = 100
embedding_dim = 8
trunc_type='post'
oov_tok = "<OOV>"

In [8]:
sequences = tokenizer.texts_to_sequences(df['processed'])
padded_sequences = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

In [9]:
padded_sequences.shape

(1503, 100)

In [10]:
padded_sequences[1]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,    63,  2175,  7429,  4722, 12385,  5564,     1,    36,
          36])

In [11]:
predictions = model.predict(padded_sequences)

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [12]:
# Menginterpretasikan hasil prediksi
# Misalnya, mengasumsikan kelas 0 sebagai "Netral", kelas 1 sebagai "Negatif", dan kelas 2 sebagai "Positif"
predicted_labels = []
predicted_probabilities = []  # Menyimpan probabilitas prediksi untuk semua kelas

for pred in predictions:
    # Menentukan kelas berdasarkan nilai probabilitas tertinggi
    label_index = np.argmax(pred)
    if label_index == 0:
        predicted_labels.append("Netral")
    elif label_index == 1:
        predicted_labels.append("Negatif")
    else:
        predicted_labels.append("Positif")
    
    # Menyimpan probabilitas prediksi untuk semua kelas
    predicted_probabilities.append(pred)  # Menyimpan array probabilitas dari softmax

# Menambahkan hasil prediksi dan probabilitas prediksi ke dalam data asli
df['predicted_label'] = predicted_labels
df['predicted_probability_netral'] = [prob[0] for prob in predicted_probabilities]
df['predicted_probability_negatif'] = [prob[1] for prob in predicted_probabilities]
df['predicted_probability_positif'] = [prob[2] for prob in predicted_probabilities]

# Menyimpan data dengan hasil prediksi dan probabilitas prediksi ke dalam file CSV baru
df.to_csv('klasifikasi_baru_model_with_softmax_no_stemming.csv', index=False)



In [13]:
df

,full_text,processed,predicted_label,predicted_probability_netral,predicted_probability_negatif,predicted_probability_positif
0,@safergieway @PSSI True bang. bener² ga ngerti...,true bang mengerti mainnya pssi pelatih kelas ...,Negatif,0.071556,0.910904,0.017540
1,Padahal buat jam terbang dan latihan kan kalo ...,jam terbang latihan liga klub timnas mainnya t...,Negatif,0.439432,0.447716,0.112852
2,@garagarabola_ Lebih penting timnas bagus main...,timnas bagus mainnya jersey dikerenin,Positif,0.023436,0.290142,0.686422
3,@Tyows pdhal Pemain Timnas inggris nih gw rasa...,pemain timnas inggris gue enggak ada midfielde...,Negatif,0.133284,0.668104,0.198612
4,@utdfocusid @bliblidotcom Semuanya out kecuali...,out kecuali hojlund diganti pemain timnas kcua...,Netral,0.473668,0.449844,0.076488
...,...,...,...,...,...,...
1498,timnas malam ini mainnya gokil abizzzz 🇲🇨👊 #Ti...,timnas malam mainnya gila habis,Negatif,0.259735,0.690536,0.049728
1499,@Indostransfer Apresiasi utk ketahanan fisik t...,apresiasi ketahanan fisik timnas sanggup berma...,Netral,0.787389,0.125399,0.087212
1500,90 menit full deg-degan... Babak pertama mainn...,menit full degdegan babak mainnya bagus babak ...,Positif,0.007292,0.193739,0.798969
1501,"Sandy Walsh mainnya apik, solid. Beli jersey ...",sandy walsh mainnya apik solid beli jersey tim...,Negatif,0.386907,0.478620,0.134474


In [14]:
#get total number per class
df['predicted_label'].value_counts()

predicted_label
Positif    723
Negatif    617
Netral     163
Name: count, dtype: int64

In [139]:
pd.set_option('display.max_colwidth', None)

In [140]:
df[df['predicted_label'] == 'Positif'].to_csv('klasifikasi_baru_model_with_stemmed_positif.csv', index=False)

In [141]:
df[df['predicted_label'] == 'Negatif'].to_csv('klasifikasi_baru_model_with_stemmed_negatif.csv', index=False)

In [142]:
df[df['predicted_label'] == 'Netral'].to_csv('klasifikasi_baru_model_with_stemmed_netral.csv', index=False)

In [115]:
# df.to_csv('./hasil_prediksi_no_stemming.csv', index=False)

In [116]:
df_result_processing = pd.read_csv('./klasifikasi_baru_model_with_stemmed.csv', sep=",", header=None, skiprows=1)
df_result_no_processing = pd.read_csv('./klasifikasi_baru_model_no_stemmed.csv', sep=",", header=None, skiprows=1)

# df_result_processing.columns = ["full_text", "processed", "predicted_labels"]
# df_result_no_processing.columns = ["full_text", "processed" "predicted_labels"]

FileNotFoundError: [Errno 2] No such file or directory: './klasifikasi_baru_model_with_stemmed.csv'

In [ ]:
df_result_processing.head()

,0,1,2,3
0,"@safergieway @PSSI True bang. bener² ga ngerti sih cara mainnya pssi, pelatih kelas dunia mah bisa kapan ajaa. kalo kompetisi ga jalan yaa percuma wkwk timnas cowok aja yg punya kompetisi setengah mati pengen prestasi",true bang erti main pssi latih kelas dunia mah kompetisi jalan wkwk timnas cowok kompetisi mati pengin prestasi,Positif,0.544604
1,Padahal buat jam terbang dan latihan kan kalo ada liga dan klub ya timnas aja mainnya per berapa tahun dl 😅,jam terbang latih liga klub timnas main,Positif,0.682539
2,@garagarabola_ Lebih penting timnas bagus mainnya daripada segala Jersey aja dikerenin🤭,timnas bagus main jersey renin,Negatif,0.184601
3,"@Tyows pdhal Pemain Timnas inggris nih gw rasa gada 1 pun Midfielder yg gaya mainnya kyak Mainoo,kenapa masih ga sat set buat manggil dia wkwk",main timnas inggris gue enggak ada midfielder gaya main kayak mainookenapa sat set manggil wkwk,Positif,0.935572
4,"@utdfocusid @bliblidotcom Semuanya out kecuali hojlund, diganti sm pemain2 timnas kcuali Struick wkwk mainnya masih bagusan timnas kmrn cok",out kecuali hojlund ganti main timnas kcuali struick wkwk main bagus timnas kemarin cok,Negatif,0.356904


In [ ]:
df_result_no_processing.head()

,0,1,2
0,"@safergieway @PSSI True bang. bener² ga ngerti sih cara mainnya pssi, pelatih kelas dunia mah bisa kapan ajaa. kalo kompetisi ga jalan yaa percuma wkwk timnas cowok aja yg punya kompetisi setengah mati pengen prestasi",true bang mengerti mainnya pssi pelatih kelas dunia mah kompetisi jalan wkwk timnas cowok kompetisi mati pengin prestasi,Positif
1,Padahal buat jam terbang dan latihan kan kalo ada liga dan klub ya timnas aja mainnya per berapa tahun dl 😅,jam terbang latihan liga klub timnas mainnya tahun-tahun,Positif
2,@garagarabola_ Lebih penting timnas bagus mainnya daripada segala Jersey aja dikerenin🤭,timnas bagus mainnya jersey dikerenin,Positif
3,"@Tyows pdhal Pemain Timnas inggris nih gw rasa gada 1 pun Midfielder yg gaya mainnya kyak Mainoo,kenapa masih ga sat set buat manggil dia wkwk",pemain timnas inggris gue enggak ada midfielder gaya mainnya kayak mainookenapa sat set manggil wkwk,Positif
4,"@utdfocusid @bliblidotcom Semuanya out kecuali hojlund, diganti sm pemain2 timnas kcuali Struick wkwk mainnya masih bagusan timnas kmrn cok",out kecuali hojlund diganti pemain timnas kcuali struick wkwk mainnya bagusan timnas kemarin cok,Positif


In [ ]:
jumlah_positive_processing = (df_result_processing[2] == "Positif").sum()
print(f'Jumlah review positif: {jumlah_positive_processing}')

jumlah_negatif_processing = (df_result_processing[2] == "Negatif").sum()
print(f'Jumlah review negatif: {jumlah_negatif_processing}')

Jumlah review positif: 1106
Jumlah review negatif: 397


In [ ]:
jumlah_positive_no_processing = (df_result_no_processing[2] == "Positif").sum()
print(f'Jumlah review positif: {jumlah_positive_no_processing}')

jumlah_negatif_no_processing = (df_result_no_processing[2] == "Negatif").sum()
print(f'Jumlah review negatif: {jumlah_negatif_no_processing}')

Jumlah review positif: 1106
Jumlah review negatif: 397
